<h2 style="text-align:center;font-size:200%;;">Лабораторная работа №1 </h2>
<h3  style="text-align:center;"><span class="label label-success">Проведение разведочного анализа датасета</span>


1. Initialize pyspark framework and load data into pyspark's dataframe

In [1]:
# Import other modules not related to PySpark
import os
import sys
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
import math
import json
from IPython.core.interactiveshell import InteractiveShell
from datetime import *
import statistics as stats
# This helps auto print out the items without explixitly using 'print'
InteractiveShell.ast_node_interactivity = "all" 
%matplotlib inline

# Import PySpark related modules
import pyspark
from pyspark.rdd import RDD
from pyspark.sql import Row
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions
from pyspark.sql.functions import lit, desc, col, size, array_contains\
, isnan, udf, hour, array_min, array_max, countDistinct
from pyspark.sql.types import *

MAX_MEMORY = '15G'
# Initialize a spark session.
conf = pyspark.SparkConf().setMaster("local[*]") \
        .set('spark.executor.heartbeatInterval', 10000) \
        .set('spark.network.timeout', 10000) \
        .set("spark.core.connection.ack.wait.timeout", "3600") \
        .set("spark.executor.memory", MAX_MEMORY) \
        .set("spark.driver.memory", MAX_MEMORY)
def init_spark():
    spark = SparkSession \
        .builder \
        .appName("SOBD2023") \
        .config(conf=conf) \
        .getOrCreate()
    return spark

spark = init_spark()
filename_data = 'dataindiv/14.csv'
# Загружаем датасет,выводим первые 5 строк 
df = spark.read \
    .option("header", True) \
    .option("delimiter", ";") \
    .csv("dataindiv/8.csv")


# Далее для удобства берем только первые 8 столбцов и определяем тип их данных

df = df["timestamp", "actual_consumption", "load_00", "load_01", "load_02", "load_03", "load_04", "load_05"]
df.show()

print('Data frame type: ' + str(type(df)))

+--------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|           timestamp|actual_consumption|           load_00|           load_01|           load_02|           load_03|           load_04|           load_05|
+--------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|2015-06-12T02:00:...| 28.85647308677435|26.481838220094716|23.421641718692797| 20.57187549747037|  20.2664084859748|18.748720578799485| 19.16973657657757|
|2015-06-12T02:15:...|26.465026293136184| 24.16895244093518|21.870495741922895|  20.2664084859748|18.748720578799485| 19.16973657657757|18.706966825755106|
|2015-06-12T02:45:...| 20.25967533160001| 18.89810303243156|16.316711645913593| 19.16973657657757|18.706966825755106|18.683137459003262|18.713580181675162|
|2015-06-12T03:30:...| 34.02441665679216|30.685165498424723|27.7

2. Обзор датасета

In [2]:
print('Data overview')
df.printSchema()
print('Columns overview')
pd.DataFrame(df.dtypes, columns = ['Column Name','Data type'])

Data overview
root
 |-- timestamp: string (nullable = true)
 |-- actual_consumption: string (nullable = true)
 |-- load_00: string (nullable = true)
 |-- load_01: string (nullable = true)
 |-- load_02: string (nullable = true)
 |-- load_03: string (nullable = true)
 |-- load_04: string (nullable = true)
 |-- load_05: string (nullable = true)

Columns overview


,Column Name,Data type
0,timestamp,string
1,actual_consumption,string
2,load_00,string
3,load_01,string
4,load_02,string
5,load_03,string
6,load_04,string
7,load_05,string


Как мы видим, тип данных определился неправильно. Соответственно, необходимо восстановить исходные типы данных

In [3]:
from pyspark.sql.types import FloatType, IntegerType
df = df.withColumn("actual_consumption", df.actual_consumption.cast(IntegerType()))
df = df.withColumn("load_00", df.load_00.cast(FloatType()))
df = df.withColumn("load_01", df.load_01.cast(FloatType()))
df = df.withColumn("load_02", df.load_02.cast(FloatType()))
df = df.withColumn("load_03", df.load_03.cast(FloatType()))
df = df.withColumn("load_04", df.load_04.cast(FloatType()))
df = df.withColumn("load_05", df.load_05.cast(FloatType()))

Проверим значения еще раз

In [4]:
print('Data overview')
df.printSchema()
print('Columns overview')
pd.DataFrame(df.dtypes, columns = ['Column Name','Data type'])

Data overview
root
 |-- timestamp: string (nullable = true)
 |-- actual_consumption: integer (nullable = true)
 |-- load_00: float (nullable = true)
 |-- load_01: float (nullable = true)
 |-- load_02: float (nullable = true)
 |-- load_03: float (nullable = true)
 |-- load_04: float (nullable = true)
 |-- load_05: float (nullable = true)

Columns overview


,Column Name,Data type
0,timestamp,string
1,actual_consumption,int
2,load_00,float
3,load_01,float
4,load_02,float
5,load_03,float
6,load_04,float
7,load_05,float


3. Поиск пропущенных значений

In [5]:
print('Columns overview')
pd.DataFrame(df.dtypes, columns = ['Column Name','Data type'])

Columns overview


,Column Name,Data type
0,timestamp,string
1,actual_consumption,int
2,load_00,float
3,load_01,float
4,load_02,float
5,load_03,float
6,load_04,float
7,load_05,float


In [6]:
numeric_columns = ['timestamp', 'actual_consumption', 'load_00',"load_01","load_02","load_03", "load_04", "load_05"]
string_columns = ["timestamp"]
missing_values = {}
for index, column in enumerate(df.columns):
    if column in string_columns:    # check string columns with None and Null values
#         missing_count = df.filter(col(column).eqNullSafe(None) | col(column).isNull()).count()
#         missing_values.update({column: missing_count})
        missing_count = df.filter(col(column).eqNullSafe(None) | col(column).isNull()).count()
        missing_values.update({column:missing_count})
    if column in numeric_columns:  # check zeroes, None, NaN
        missing_count = df.where(col(column).isin([None,np.nan])).count()
        missing_values.update({column:missing_count})
missing_df = pd.DataFrame.from_dict([missing_values])
missing_df


,timestamp,actual_consumption,load_00,load_01,load_02,load_03,load_04,load_05
0,0,0,0,0,0,0,0,0


4. Поиск выбросов и их устранение

In [7]:
# Вычисляем Q1 и Q3 для actual_consumption
Q1 = df.approxQuantile("actual_consumption", [0.25], 0)[0]
Q3 = df.approxQuantile("actual_consumption", [0.75], 0)[0]

# Вычисляем IQR
IQR = Q3 - Q1

# Определяем границы выбросов
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Создаем новый столбец с замененными выбросами
df = df.withColumn("new_value", when(col("actual_consumption") < lower_bound, Q1).when(col("actual_consumption") > upper_bound, Q3).otherwise(col("actual_consumption")))

# Удаляем старый столбец и переименовываем новый
df = df.drop("actual_consumption").withColumnRenamed("new_value", "actual_consumption")

# Теперь повторряем эту операцию для каждого столбца

Q1 = df.approxQuantile("load_00", [0.25], 0)[0]
Q3 = df.approxQuantile("load_00", [0.75], 0)[0]
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
df = df.withColumn("new_value", when(col("load_00") < lower_bound, Q1).when(col("load_00") > upper_bound, Q3).otherwise(col("load_00")))
df = df.drop("load_00").withColumnRenamed("new_value", "load_00")

Q1 = df.approxQuantile("load_01", [0.25], 0)[0]
Q3 = df.approxQuantile("load_01", [0.75], 0)[0]
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
df = df.withColumn("new_value", when(col("load_01") < lower_bound, Q1).when(col("load_01") > upper_bound, Q3).otherwise(col("load_01")))
df = df.drop("load_01").withColumnRenamed("new_value", "load_01")

Q1 = df.approxQuantile("load_02", [0.25], 0)[0]
Q3 = df.approxQuantile("load_02", [0.75], 0)[0]
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
df = df.withColumn("new_value", when(col("load_02") < lower_bound, Q1).when(col("load_02") > upper_bound, Q3).otherwise(col("load_02")))
df = df.drop("load_02").withColumnRenamed("new_value", "load_02")

Q1 = df.approxQuantile("load_03", [0.25], 0)[0]
Q3 = df.approxQuantile("load_03", [0.75], 0)[0]
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
df = df.withColumn("new_value", when(col("load_03") < lower_bound, Q1).when(col("load_03") > upper_bound, Q3).otherwise(col("load_03")))
df = df.drop("load_03").withColumnRenamed("new_value", "load_03")

Q1 = df.approxQuantile("load_04", [0.25], 0)[0]
Q3 = df.approxQuantile("load_04", [0.75], 0)[0]
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
df = df.withColumn("new_value", when(col("load_04") < lower_bound, Q1).when(col("load_04") > upper_bound, Q3).otherwise(col("load_04")))
df = df.drop("load_04").withColumnRenamed("new_value", "load_04")

Q1 = df.approxQuantile("load_05", [0.25], 0)[0]
Q3 = df.approxQuantile("load_05", [0.75], 0)[0]
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
df = df.withColumn("new_value", when(col("load_05") < lower_bound, Q1).when(col("load_05") > upper_bound, Q3).otherwise(col("load_05")))
df = df.drop("load_05").withColumnRenamed("new_value", "load_05")

df.show()

NameError: name 'when' is not defined

5. Вычисляем статистические показатели

In [ ]:
# Вычисляем статистики для столбцов "actual_consumption", "load_00", "load_01", "load_02", "load_03", "load_04", "load_05"
stats = df.select("actual_consumption", "load_00", "load_01", "load_02", "load_03", "load_04", "load_05").summary()
stats.show()

6. Визуализация распределения наиболее важных признаков

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Выбираем признаки
important_cols = ["actual_consumption", "load_00", "load_01", "load_02", "load_03", "load_04", "load_05"]

# Создаем новый датафрейм с выбранными признаками
df_selected = df.select(important_cols)

# Преобразуем датафрейм PySpark в датафрейм Pandas
df_pd = df_selected.toPandas()

# Строим гистограммы для каждого признака
fig, axes = plt.subplots(nrows=1, ncols=len(important_cols), figsize=(10, 5))
for i, col in enumerate(important_cols):
    ax = axes[i]
    ax.hist(df_pd[col], bins=20)
    ax.set_title(col)
plt.show()

In [ ]:
from pyspark.sql.functions import corr
# Просматриваем первые пять строк датасета
df.show(5)

# Вычисляем корреляцию Пирсона между колонками "load_00" и "load_01"
pearson_corr = df.corr("actual_consumption", "load_01", method="pearson")
print(f"Корреляция Пирсона между actual_consumption и load_01: {pearson_corr}")

# Вычисляем корреляцию Пирсона между колонками "load_02" и "load_03"
pearson_corr = df.corr("actual_consumption", "load_03", method="pearson")
print(f"Корреляция Пирсона между actual_consumption и load_01: {pearson_corr}")

# Вычисляем корреляцию Пирсона между колонками "load_04" и "load_05"
pearson_corr = df.corr("actual_consumption", "load_05", method="pearson")
print(f"Корреляция Пирсона между actual_consumption и load_01: {pearson_corr}")
